In [ ]:
# this function is going to remove outliers in the concentration gradient of LiFSI G4 electrolyte
# mark outliers as "NaH" value in the arrays
# so the new plot won't show the outliers anymore
# if you have some problem， contact me via junyi.zhao@stx.ox.ac.uk

# An example of calling this function
#getcurvefitting(Z_distance,eva_dictionary,total file number)

from copy import deepcopy
def removeoutliers(z7,e,totalfilenumber):
    
    import numpy             as np
    from lmfit       import Model, Parameters, minimize, fit_report
    from scipy       import integrate, special
    import matplotlib.cm     as cm
    import matplotlib.pyplot as plt
    from cycler      import cycler
    e_good=deepcopy(e)

    k = 0
    for i in range(totalfilenumber):
        x=np.array(e[i][0])
        ydata=np.array(e[i][1])
        c7=sum(ydata)/50
        def func(x, a, b):return  (a)*((b/(3.1415)**0.5)*np.exp(-((((-x+z7)/1e6)/b)**2))-((((-x+z7)/1e6))*special.erfc(((-x+z7)/1e6)/b)))-(a)*((b/(3.1415)**0.5)*np.exp(-(((((x)/1e6))/b)**2))-((((x)/1e6))*special.erfc(((x)/1e6)/b)))+c7

        #lmfit function described above
        gmodel = Model(func)
        params = Parameters()
        params.add('a', value=-184000, vary=True)#min=100000, max=300000)
        params.add('b', value=0.004, vary=True, min=1e-6, max=1e-1)
        result = gmodel.fit(ydata, params, x=x, weights=np.sqrt(1.0/ydata))
        
        #plot
        plt.plot(x, result.best_fit, label = "Fitted Curve"+str(i))
        plt.xlabel('Distance / um')
        plt.ylabel('Concentration / $molkg^-3$')
        plt.title('Hypothetical fit for removing outliers')
        
        #l2=list(range(0*4,totalfilenumber*4,4))
        #plt.legend(l2,fontsize=5)
        #plt.ylim([0.2,1.4])
        plt.xlim([0,18000])

        fake_a=result.params['a'].value
        fake_b=result.params['b'].value

        #fake_f=(x,fake_a,fake_b)

        for j in range(0,50,1):
            
            if e[i][1][j]-func(e[i][0][j],fake_a,fake_b)> 0.16 or e[i][1][j]-func(e[i][0][j],fake_a,fake_b)<-0.16:
                
                e_good[i][1][j]=np.nan
        
    return e_good
